In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

In [2]:
user_info = pd.read_csv("/content/drive/MyDrive/CECS550/data_format1/user_info_format1.csv");
user_log = pd.read_csv("/content/drive/MyDrive/CECS550/data_format1/user_log_format1.csv");
train_format = pd.read_csv("/content/drive/MyDrive/CECS550/data_format1/train_format1.csv");


In [3]:
user_log = user_log[(user_log['item_id']>=641) & (user_log['item_id'] <=800)]
# user_log.to_csv("/content/drive/MyDrive/CECS550/data_format1/user_log_modified.csv",index=False)

In [4]:
user_info.columns

Index(['user_id', 'age_range', 'gender'], dtype='object')

In [5]:
user_log.columns

Index(['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id', 'time_stamp',
       'action_type'],
      dtype='object')

In [6]:
train_format.columns

Index(['user_id', 'merchant_id', 'label'], dtype='object')

In [7]:
user_log = user_log.rename(columns={'seller_id': 'merchant_id'})

In [8]:
data = pd.merge(user_log, user_info, on='user_id', how='left')
train_format.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [9]:
data = pd.merge(data, train_format, on=['user_id', 'merchant_id'])
print(data.head())

   user_id  item_id  cat_id  merchant_id  brand_id  time_stamp  action_type  \
0   310303      650     656         3473    3305.0        1111            0   
1   310303      650     656         3473    3305.0        1111            0   
2   310303      650     656         3473    3305.0        1111            2   
3   310303      650     656         3473    3305.0        1111            0   
4   156939      653      35         4267    6046.0        1111            0   

   age_range  gender  label  
0        4.0     0.0      0  
1        4.0     0.0      0  
2        4.0     0.0      0  
3        4.0     0.0      0  
4        4.0     1.0      1  


In [10]:
data.shape

(449, 10)

In [11]:
data['Day']=data['time_stamp']%100
data['Month']=(data['time_stamp']-data['Day'])/100
data = data.drop(columns = ['time_stamp'])

In [12]:
#activity_count of user
activity_count = data.groupby('user_id')['action_type'].count().reset_index()
activity_count.columns = ['user_id', 'activity_count']
data = pd.merge(data, activity_count, on='user_id', how='left')

In [13]:
data.shape

(449, 12)

In [14]:
click_count = data[data['action_type']==0].groupby('user_id')['action_type'].count().reset_index()
click_count.columns = ['user_id', 'click_count']
data = pd.merge(data,click_count,on='user_id', how='left')

In [15]:
data.shape

(449, 13)

In [16]:
add_to_cart_count = data[data['action_type']==1].groupby('user_id')['action_type'].count().reset_index()
add_to_cart_count.columns = ['user_id', 'add_to_cart_count']
data = pd.merge(data, add_to_cart_count, on='user_id', how='left')

In [17]:
data.shape

(449, 14)

In [18]:
purchase_count = data[data['action_type']==2].groupby('user_id')['action_type'].count().reset_index()
purchase_count.columns = ['user_id', 'purchase_count']
data = pd.merge(data, purchase_count, on='user_id', how='left')

In [19]:
data.shape

(449, 15)

In [20]:
favourite_count = data[data['action_type']==3].groupby('user_id')['action_type'].count().reset_index()
favourite_count.columns = ['user_id', 'favourite_count']
data = pd.merge(data, favourite_count, on='user_id', how='left')

In [21]:
data.shape

(449, 16)

In [22]:
data[['click_count','activity_count','add_to_cart_count','purchase_count','favourite_count']] = data[['click_count','activity_count','add_to_cart_count','purchase_count','favourite_count']].fillna(0)

In [23]:
data['gender'] = data['gender'].fillna(2)

In [24]:
user_gender = label_binarize(np.array(data.gender), classes=[0,1,2])
user_gender = pd.DataFrame(data=user_gender, columns=['gender_female','gender_male','gender_unknown'])
data['user_gender_female']=user_gender['gender_female']
data['user_gender_male']=user_gender['gender_male']
data['user_gender_unknown']=user_gender['gender_unknown']
data = data.drop(columns = ['gender'])
data.head()

,user_id,item_id,cat_id,merchant_id,brand_id,action_type,age_range,label,Day,Month,activity_count,click_count,add_to_cart_count,purchase_count,favourite_count,user_gender_female,user_gender_male,user_gender_unknown
0,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,4,3.0,0.0,1.0,0.0,1,0,0
1,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,4,3.0,0.0,1.0,0.0,1,0,0
2,310303,650,656,3473,3305.0,2,4.0,0,11,11.0,4,3.0,0.0,1.0,0.0,1,0,0
3,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,4,3.0,0.0,1.0,0.0,1,0,0
4,156939,653,35,4267,6046.0,0,4.0,1,11,11.0,2,1.0,0.0,1.0,0.0,0,1,0


In [25]:
data.shape

(449, 18)

In [26]:
user_activity = data[['user_id','action_type']].copy()
user_activity_info = label_binarize(np.array(data.action_type), classes=[0,1,2,3])
user_activity_info = pd.DataFrame(data = user_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
user_activity['user_Click']= user_activity_info.Click
user_activity['user_Add_to_cart']=user_activity_info.add_to_cart
user_activity['user_purchase']=user_activity_info.purchase
user_activity['user_Add_to_favorite']=user_activity_info.add_to_favorite

data = pd.merge(data, user_activity, on=['user_id','action_type'], how='left')
data.head()

,user_id,item_id,cat_id,merchant_id,brand_id,action_type,age_range,label,Day,Month,...,add_to_cart_count,purchase_count,favourite_count,user_gender_female,user_gender_male,user_gender_unknown,user_Click,user_Add_to_cart,user_purchase,user_Add_to_favorite
0,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,...,0.0,1.0,0.0,1,0,0,1,0,0,0
1,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,...,0.0,1.0,0.0,1,0,0,1,0,0,0
2,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,...,0.0,1.0,0.0,1,0,0,1,0,0,0
3,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,...,0.0,1.0,0.0,1,0,0,1,0,0,0
4,310303,650,656,3473,3305.0,0,4.0,0,11,11.0,...,0.0,1.0,0.0,1,0,0,1,0,0,0


In [27]:
data.fillna(0,inplace=True)

In [28]:
data.shape

(3309, 22)

In [29]:
data['age_range'] = data['age_range'].replace(8,7)
age =  pd.DataFrame(pd.get_dummies(data['age_range']))
column_name = {3.0:'age_25_29',
4.0:'age_30_34',
0.0:'age_0_18',
2.0:'age_18_24',
6.0:'age_40_49',
5.0:'age_35_39'}
age = age.rename(columns=column_name)

In [30]:
for val in age.columns:
  data[val] = age[val]
data = data.drop(columns=['age_range'])

In [31]:
data.shape

(3309, 27)

In [32]:
data.head().T

,0,1,2,3,4
user_id,310303.0,310303.0,310303.0,310303.0,310303.0
item_id,650.0,650.0,650.0,650.0,650.0
cat_id,656.0,656.0,656.0,656.0,656.0
merchant_id,3473.0,3473.0,3473.0,3473.0,3473.0
brand_id,3305.0,3305.0,3305.0,3305.0,3305.0
action_type,0.0,0.0,0.0,0.0,0.0
label,0.0,0.0,0.0,0.0,0.0
Day,11.0,11.0,11.0,11.0,11.0
Month,11.0,11.0,11.0,11.0,11.0
activity_count,4.0,4.0,4.0,4.0,4.0


<h2>Seller Level Info<h2>

We try to find new features with respect to different criteria i.e the merchant having interaction with item,brand,cateogry,

In [33]:

merchant_item = data.loc[:, ['merchant_id', 'item_id']]

merchant_item = merchant_item.drop_duplicates(subset='item_id')

merchant_item_info = merchant_item.groupby('merchant_id').agg({'item_id': 'count'}).reset_index()

merchant_item_info = merchant_item_info.rename(columns={'item_id': 'merchant_item_count'})

print(merchant_item_info.head(50))

data = pd.merge(data, merchant_item_info, on='merchant_id', how='left')


    merchant_id  merchant_item_count
0           252                    1
1           350                    1
2           412                    1
3           523                    1
4           586                    1
5           630                    1
6           643                    1
7           804                    1
8          1429                    1
9          1472                    1
10         1501                    1
11         1840                    1
12         1841                    1
13         1944                    1
14         2242                    1
15         3432                    1
16         3473                    1
17         3502                    1
18         3828                    1
19         4267                    1
20         4502                    1
21         4534                    1
22         4847                    1
23         4850                    1


In [34]:

merchant_brand = data.loc[:, ['merchant_id', 'brand_id']]
merchant_brand = merchant_brand.drop_duplicates(subset='brand_id')
merchant_brand_info = merchant_brand.groupby('merchant_id').agg({'brand_id': 'count'}).reset_index()
merchant_brand_info = merchant_brand_info.rename(columns={'brand_id': 'merchant_brand_count'})
print(merchant_brand_info.head(50))

data = pd.merge(data, merchant_brand_info, on='merchant_id', how='left')

    merchant_id  merchant_brand_count
0           252                     1
1           350                     1
2           412                     1
3           523                     1
4           586                     1
5           630                     1
6           643                     1
7           804                     1
8          1429                     1
9          1472                     1
10         1501                     1
11         1840                     1
12         1841                     1
13         1944                     1
14         2242                     1
15         3432                     1
16         3473                     1
17         3502                     1
18         3828                     1
19         4267                     1
20         4502                     1
21         4534                     1
22         4847                     2
23         4850                     1


<h2>User Seller Info<h2>

In [35]:
merchant_cat = data.loc[:, ['merchant_id', 'cat_id']]
merchant_cat_info = merchant_cat.groupby('merchant_id').agg({'cat_id': 'count'}).reset_index()
merchant_cat_info = merchant_cat_info.rename(columns={'cat_id': 'merchant_cat_count'})
print(merchant_cat_info.head(50))
data = pd.merge(data, merchant_cat_info, on='merchant_id', how='left')

    merchant_id  merchant_cat_count
0           252                   1
1           350                 232
2           412                   1
3           523                   1
4           586                  11
5           630                  51
6           643                   5
7           804                  28
8          1429                   1
9          1472                  10
10         1501                  15
11         1840                   7
12         1841                   1
13         1944                   1
14         2242                   1
15         3432                   1
16         3473                2460
17         3502                   1
18         3828                   3
19         4267                 366
20         4502                   1
21         4534                  55
22         4847                  39
23         4850                  17


In [36]:
user_merchant = data.loc[:, ['user_id','merchant_id']]
user_merchant = user_merchant.drop_duplicates(subset='merchant_id')
user_merchant_info = user_merchant.groupby('user_id').agg({'merchant_id': 'count'}).reset_index()
user_merchant_info = user_merchant_info.rename(columns={'merchant_id': 'user_merchant_count'})
print(user_merchant_info.head(50))
data = pd.merge(data, user_merchant_info, on='user_id', how='left')

    user_id  user_merchant_count
0     14196                    1
1     54738                    1
2     56665                    1
3     63183                    1
4     80940                    1
5     87997                    1
6     92655                    1
7    112081                    1
8    112950                    1
9    148947                    1
10   156939                    1
11   156965                    1
12   182603                    1
13   242112                    1
14   264378                    1
15   310303                    1
16   313743                    1
17   316744                    1
18   327778                    1
19   329471                    1
20   329675                    1
21   337468                    1
22   355457                    1
23   413107                    1


In [37]:
user_cat = data.loc[:, ['user_id','cat_id']]
user_cat = user_cat.drop_duplicates(subset='cat_id')
user_cat_info = user_cat.groupby('user_id').agg({'cat_id': 'count'}).reset_index()
user_cat_info = user_cat_info.rename(columns={'cat_id': 'user_cat_count'})
print(user_cat_info.head(50))
data = pd.merge(data, user_cat_info, on='user_id', how='left')

    user_id  user_cat_count
0     14196               1
1     54738               1
2     56665               1
3     63183               1
4     80940               1
5     87997               1
6     92655               1
7    112950               1
8    156939               1
9    156965               1
10   182603               1
11   242112               1
12   310303               1
13   313743               1
14   316744               1
15   327778               1
16   329471               1
17   337468               1
18   355457               1
19   413107               1


In [38]:
user_brand = data.loc[:, ['user_id','brand_id']]
user_brand = user_brand.drop_duplicates(subset='brand_id')
user_brand_info = user_brand.groupby('user_id').agg({'brand_id': 'count'}).reset_index()
user_brand_info = user_brand_info.rename(columns={'brand_id': 'user_brand_count'})
print(user_brand_info.head(50))
data = pd.merge(data, user_brand_info, on='user_id', how='left')

    user_id  user_brand_count
0     14196                 1
1     54738                 1
2     56665                 1
3     63183                 1
4     80940                 1
5     87997                 1
6     92655                 1
7    112081                 1
8    112950                 1
9    148947                 1
10   156939                 1
11   156965                 2
12   182603                 1
13   242112                 1
14   264378                 1
15   310303                 1
16   313743                 1
17   316744                 1
18   327778                 1
19   329471                 1
20   329675                 1
21   337468                 1
22   355457                 1
23   413107                 1


<h2>User Seller Merchant Info<h2>

In [39]:
user_brand = data.loc[:, ['user_id','merchant_id','brand_id']]
user_brand_info = user_brand.groupby(['user_id','merchant_id']).agg({'brand_id': 'count'}).reset_index()
user_brand_info = user_brand_info.rename(columns={'brand_id': 'user_merchant_brand_count'})
print(user_brand_info.head(50))
data = pd.merge(data, user_brand_info, on=['user_id','merchant_id'], how='left')

    user_id  merchant_id  user_merchant_brand_count
0      6308         4267                          9
1     14196         4850                          1
2     15333         3473                          2
3     21456         4267                          4
4     22289         4267                          1
5     30637         4267                         49
6     31008          643                          1
7     39322         1501                          1
8     40560         3473                        198
9     42310         4850                         16
10    42460         4267                          4
11    43489         4267                          1
12    44525         4534                          1
13    47962          804                          4
14    54159         3473                         26
15    54738          586                          9
16    54820         1501                          1
17    55308          804                          4
18    56282 

In [40]:
user_merchant_cat = data.loc[:, ['user_id','merchant_id','cat_id']]
user_merchant_cat_info = user_merchant_cat.groupby(['user_id','merchant_id']).agg({'cat_id': 'count'}).reset_index()
user_merchant_cat_info = user_merchant_cat_info.rename(columns={'cat_id': 'user_merchant_cat_count'})
print(user_merchant_cat_info.head(50))
data = pd.merge(data, user_merchant_cat_info, on=['user_id','merchant_id'], how='left')

    user_id  merchant_id  user_merchant_cat_count
0      6308         4267                        9
1     14196         4850                        1
2     15333         3473                        2
3     21456         4267                        4
4     22289         4267                        1
5     30637         4267                       49
6     31008          643                        1
7     39322         1501                        1
8     40560         3473                      198
9     42310         4850                       16
10    42460         4267                        4
11    43489         4267                        1
12    44525         4534                        1
13    47962          804                        4
14    54159         3473                       26
15    54738          586                        9
16    54820         1501                        1
17    55308          804                        4
18    56282         4847                        1


In [41]:
user_merchant_day_cat = data.loc[:, ['user_id','merchant_id','Day','cat_id']]
user_merchant_day_cat_info = user_merchant_day_cat.groupby(['user_id','merchant_id','Day']).agg({'cat_id': 'count'}).reset_index()
user_merchant_day_cat_info = user_merchant_day_cat_info.rename(columns={'cat_id': 'user_merchant_day_cat_count'})


data = pd.merge(data, user_merchant_day_cat_info, on=['user_id','merchant_id','Day'], how='left')

print(user_merchant_day_cat_info.head(10))

   user_id  merchant_id  Day  user_merchant_day_cat_count
0     6308         4267   10                            9
1    14196         4850   28                            1
2    15333         3473   11                            2
3    21456         4267   11                            4
4    22289         4267   11                            1
5    30637         4267   10                           14
6    30637         4267   11                           35
7    31008          643   10                            1
8    39322         1501   10                            1
9    40560         3473    8                           56


In [42]:
data.shape

(3309, 36)

In [43]:
data.columns

Index(['user_id', 'item_id', 'cat_id', 'merchant_id', 'brand_id',
       'action_type', 'label', 'Day', 'Month', 'activity_count', 'click_count',
       'add_to_cart_count', 'purchase_count', 'favourite_count',
       'user_gender_female', 'user_gender_male', 'user_gender_unknown',
       'user_Click', 'user_Add_to_cart', 'user_purchase',
       'user_Add_to_favorite', 'age_0_18', 'age_18_24', 'age_25_29',
       'age_30_34', 'age_35_39', 'age_40_49', 'merchant_item_count',
       'merchant_brand_count', 'merchant_cat_count', 'user_merchant_count',
       'user_cat_count', 'user_brand_count', 'user_merchant_brand_count',
       'user_merchant_cat_count', 'user_merchant_day_cat_count'],
      dtype='object')

In [44]:
len(data)

3309

In [45]:
numerical_features = ['activity_count', 'click_count',
       'add_to_cart_count', 'purchase_count', 'favourite_count','merchant_item_count',
        'merchant_brand_count', 'merchant_cat_count', 'user_merchant_count',
        'user_cat_count', 'user_brand_count', 'user_merchant_brand_count',
        'user_merchant_cat_count', 'user_merchant_day_cat_count']

In [46]:
numerical_data = data[numerical_features]

In [47]:
categorical_features = list(set(data.columns)- set(numerical_features))

In [48]:
categorical_data = data[categorical_features]

In [49]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,3309.0,224021.426111,114539.204219,6308.0,135595.0,196029.0,327778.0,414003.0
item_id,3309.0,660.195527,28.645514,644.0,650.0,650.0,653.0,798.0
cat_id,3309.0,634.822907,311.445412,35.0,656.0,656.0,656.0,1577.0
merchant_id,3309.0,3279.944394,1019.598601,252.0,3473.0,3473.0,3473.0,4850.0
brand_id,3309.0,3586.232699,1059.355053,799.0,3305.0,3305.0,3305.0,7863.0
action_type,3309.0,0.045331,0.323972,0.0,0.0,0.0,0.0,3.0
label,3309.0,0.011484,0.106562,0.0,0.0,0.0,0.0,1.0
Day,3309.0,11.218495,4.721091,1.0,10.0,11.0,11.0,31.0
Month,3309.0,10.867029,0.577567,5.0,11.0,11.0,11.0,11.0
activity_count,3309.0,14.993351,6.637073,1.0,9.0,16.0,21.0,24.0


In [50]:
data.to_csv (r'/content/drive/MyDrive/CECS550/data_format1/train_complete_features.csv', index = False, header=True)